# Python basics EXTRA

## Comprehensions and generators

We have already had a sneak peak of comprehensions, but here we explain it more in detail.

*Comprehensions* are very useful to make code cleaner and easier to read. Let us say we have a function that determines whether a number is a prime number. (This function is very inefficient, so don't "do this at home.") If there's anything in this function that is unclear, don't worry. We'll get to it.

In [164]:
import math

def is_prime(number):
    return number > 1 and all(number % divisor != 0 for divisor in range(2, int(math.sqrt(number) + 1)))

Let us say we want to create a list of all primes up to 20. We might be tempted to write code like this. Note the use of the `range` function to loop over integers up to a maximum (like a traditional for-loop) and the `.append()` method for lists.

In [165]:
primes = []                         # Create an empty list of prime numbers
for num in range(20):               # range(20) is the collection 0, 1, 2, ..., 19
    if is_prime(num):               # Check whether it is a prime number
        primes.append(num)          # If so, add it to the list
primes

[2, 3, 5, 7, 11, 13, 17, 19]

While this works, a much more elegant solution is the following.

In [125]:
[num for num in range(20) if is_prime(num)]

[2, 3, 5, 7, 11, 13, 17, 19]

This is called a *list comprehension*, and it's a thing of beauty. (Take a moment to reflect if you like.) The basic syntax looks like this:

`[<something> for <something> in <collection>]`

or like this:

`[<something> for <something> in <collection> if <condition>]`

Note that the condition is optional, therefore we can create a list of the numbers from 0 to 19 like this.

In [126]:
[num for num in range(20)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

Or, we could create a list of the *squares* of prime numbers like this:

In [127]:
[num**2 for num in range(20) if is_prime(num)]

[4, 9, 25, 49, 121, 169, 289, 361]

You can use comprehensions to create sets too.

In [128]:
{num for num in range(20) if is_prime(num)}

{2, 3, 5, 7, 11, 13, 17, 19}

Or even dictionaries. What do you think this does?

In [129]:
mydict = {num: is_prime(num) for num in range(20)}

You might think, then, that this creates a tuple:

In [130]:
something = (num for num in range(20) if is_prime(num))

However, this is a *generator*. A generator is a collection-like object that only creates output when requested. Therefore no primes have been computed yet. However when we loop over `something` (for example), primes appear.

In [131]:
for prime in something:
    print(prime)

2
3
5
7
11
13
17
19


If you try to loop over the same generator again, it won't work. They are one-use only.

In [132]:
for prime in something:
    print(prime)           # No output, `something` is empty

Looking back at the `is_prime` function again, we find this code:
    
    (number % divisor != 0 for divisor in range(2, int(math.sqrt(number) + 1)))
    
This is a generator that runs over all possible divisors to `number`. (The maximal possible divisor is the square root of `number`. We add one because the upper end of a `range` is exclusive, and we convert to an `int` because `range` doesn't work on floating point numbers.)

It then checks whether `number` leaves a remainder of zero when divided by `divisor`, i.e. whether `divisor` is an *actual* divisor to `number`. It then produces `False` if is is the case, or `True` if not.

A prime number is a number with no proper divisors. Therefore `number` is prime if *all* output of this generator are `True`. The function `all` checks this.

    all(number % divisor != 0 for divisor in range(2, int(math.sqrt(number) + 1))))
    
Python allows you to drop one layer of parentheses if a generator is the only argument to a function, which lets us write

    all(x for x in ...)
    
instead of

    all((x for x in ...))

## Iterables and itertools

In Python, an *iterable* is anything that can be iterated over, in other words anything that fits in a `for`-loop. Lists, tuples, dictionaries, sets and strings are all iterables, but we have seen others too: the return value of the `range` function is iterable, as are generators.

The Python ecosystem revolves heavily around iterables, and Python itself has a large amount of tools to work with them, often leading to very elegant code. I will present some of these tools here.

**WARNING:** With very few exceptions, all functions that return iterables return *generators*. In other words, they don't produce elements unless those elements are consumed by something, such as a `for`-loop. The exceptions are the functions `list`, `tuple`, `dict`, and `set`, which accept an iterable as an argument and then consumes it, returning the elements as a list, tuple, dictionary or set. Therefore, in the following, we will use `list(...)` to show the result of a piece of code. In regular code this would usually not be necessary.

The `map` function applies a function to each element of an iterable.

In [133]:
list(map(int, ['1', 2.0, 3.1]))

[1, 2, 3]

The `filter` function filters out the items of an iterable which fail a predicate test.

In [134]:
def has_length_two(s):
    return len(s) == 2

list(filter(has_length_two, ['a', 'abc', 'de', 'fg', 'hij']))

['de', 'fg']

Note that both `map` and `filter` can be expressed with comprehension syntax, and that this sort of syntax is usually considered preferable among Pythonistas.

The `enumerate` function allows you to iterate over both the elements of a collection *and* their indices at the same time.

In [135]:
for index, value in enumerate('abcd'):
    print(index, '=>', value)

0 => a
1 => b
2 => c
3 => d


This is much more elegant than code such as this:

In [136]:
s = 'abcd'
for index in range(len(s)):
    print(index, '=>', s[index])

0 => a
1 => b
2 => c
3 => d


The `zip` function lets you iterate over multiple iterables simultaneously, like a zipper.

In [137]:
list(zip('abcd', 'zyxw'))

[('a', 'z'), ('b', 'y'), ('c', 'x'), ('d', 'w')]

`zip` accepts an arbitrary number of iterables. They can even be of different length, and the total length of the iterable will be that of the shortest argument.

In [138]:
list(zip('abcd', 'zyx', 'abcdefghijkl'))

[('a', 'z', 'a'), ('b', 'y', 'b'), ('c', 'x', 'c')]

The `itertools` module contains much more goodies. Let's try some of them by importing it.

In [139]:
import itertools as it

The `product` function creates a Cartesian product of several iterables.

In [140]:
list(it.product([0, 1], 'ab'))

[(0, 'a'), (0, 'b'), (1, 'a'), (1, 'b')]

The `combinations` function returns subsets of a collection.

In [141]:
list(it.combinations('abcd', 2))

[('a', 'b'), ('a', 'c'), ('a', 'd'), ('b', 'c'), ('b', 'd'), ('c', 'd')]

The `chain` function concatenates several iterables together.

In [142]:
list(it.chain('abc', range(3)))

['a', 'b', 'c', 0, 1, 2]

The `repeat` function creates an infinite iterable that just outputs a single thing. (Don't try to do `list(repeat(...))` however.)

In [143]:
it.repeat(3)   # => 3, 3, 3, ...

repeat(3)

The `cycle` function creates an iterable that cycles through another iterable endlessly.

In [144]:
it.cycle('abc')    # => 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', ...

The `count` function creates an iterable that counts up from a given number.

In [145]:
it.count(0)     # => 0, 1, 2, 3, ...

count(0)

# Classes

[Classes](https://docs.python.org/3/tutorial/classes.html) provide a means of bundling data and functionality together. Creating a new class creates a new type of object, allowing new instances of that type to be made. Each class instance can have attributes attached to it for maintaining its state. Class instances can also have methods (defined by its class) for modifying its state.
The syntax for defining classes in Python is straightforward:

In [167]:
class Greeter(object):

    # Constructor
    def __init__(self, name):
        self.name = name  # Create an instance variable

    # Instance method
    def greet(self, loud=False):
        if loud:
            print('HELLO, %s!' % self.name.upper())
        else:
            print('Hello, %s' % self.name)

g = Greeter('Fred')  # Construct an instance of the Greeter class
g.greet()            # Call an instance method; prints "Hello, Fred"
g.greet(loud=True)   # Call an instance method; prints "HELLO, FRED!"

Hello, Fred
HELLO, FRED!


In [168]:
class Person():         # Next indented block is in the class definition

    def __init__(self, name, age):  # Values specified when object is made
        self.name = name              # Link input values to the object
        self.age = age
  
    def get_first_name(self):       # A second, custom function
        names = self.name.split()     # self refers to the run-time object
        return names[0]               # Give back first word

p1 = Person('Lisa Simpson', 8)    # Make object of Person class
p2 = Person('Bart Simpson', 10)   # Make another
print(p1.age, p2.age)             # Values linked to objects - 8, 10 
print(p1.get_first_name())        # Run a linked function - gives 'Lisa'

8 10
Lisa
